# Questions Clustering - English

## Expected
Questions to be sorted out such that the response to the whole cluster is same.

<!--### To Do-->
Author: Sunanda Bansal  
Organization: Dataperformers  
License: CC BY-NC  
Date: 24 Mar, 2020 (Start)  

In [1]:
import re
import regex
import os
import csv
import sys
import json
import time
import scipy
import socket
import pickle
import numpy as np
import pandas as pd
import multiprocessing as mp

from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import Normalizer   
from sklearn import metrics   
from sklearn.metrics import confusion_matrix
# from sklearn.metrics.pairwise import paired_distances as sklearn_paired_distances

# Plotting
import seaborn as sn
import matplotlib.pyplot as plt

# importing personal development helper classes
import utils

Using TensorFlow backend.


## Define variables here

Mostly the code will be intended to use with arguments that can be passed in comman line, but jupyter notebook doesn't handle `argparse` well, so the Args class is a temporary way to write the code assumming the variables to be an attribute of an object.

In [2]:
class Args:
    def __init__(self):
        # The very big scraped file, give absolute path, outside the repo
        # self.filename = "query_result_2020-03-27T19_12_30.866993Z.csv"
        self.filename = "covid_questions_2020-04-21.csv"
        
        # path to the file
        self.dataset = f"data/{self.filename}"    
        
        # suffix used to create
        self.suffix = "_".join([word for word in self.filename.split("_") if not word.isalpha()])[:-4]
        self.vector_mode = "tfidf"
        self.n_topics = 230
        self.dist_thresh = 1.5
        self.lang = "en"
args=Args()

In [3]:
# This dictionary is used to translate old labels to new labels minimizing modifications required for the moment
new_labels = {
                "about": "covid-what",
                "animals": "covid-animals",
                "caution": "personal-caution",
                "dangerisk": "covid-contagious",
                "diff": "covid-versus",
                "future": "situation-future",
                "guideme": "personal-whatif",
                "incubation": "covid-incubation",
                "infection": "covid-infection",
                "isolation": "personal-isolation",
                "lockdown": "situation-lockdown",
                "nextsteps": "personal-symptoms",
                "past": "situation-past",
                "recover": "covid-recovery",
                "statistics": "situation-stats",
                "symptom": "covid-symptoms",
                "test": "personal-testing",
                "transmission": "covid-transmission",
                "treatment": "covid-med",
                "unclassified": "unclassified",
                "virusfight": "covid-fight",
                "viruskill": "covid-kill",
                "viruslife": "covid-life"
            }

In [4]:
# Disable (FALSE) displaying warnings from the OpenMP* run-time library during program execution.
os.environ['KMP_WARNINGS'] = "FALSE"

## Functions

In [5]:
# Regex functions
def surrounding(word,area=2):
    # A funtion, yet to be defined, that can be used to extract text around keywords
    return

def fuzzy_match(word,pattern):
    '''
        Fuzzy matching function to be used with .apply() of pandas

        Reason - Fuzzy matching is available in regex package, not in re package, 
        therefore fuzzy matching is not a part of pandas string matching functions
    '''
 
    if regex.search(pattern, word, re.IGNORECASE):
        return True
    else:
        return False    
    
# Language Detection
from langdetect import detect
def detect_lang(text):
    # Used to detect language of the question
    try:
        return detect(text)
    except:
        return "unidentifiable"   

### Natural Lanuage Proprocessing Functions
   
Preprocessing done -
   1. Normalizing accents  
   2. Removing non alphabetic characters  
   3. Casefolding  

Preprocessing not done -
   1. Stopword removal - For questions, stopwords are essential and thus are retained
   2. Stemming - For rule based analysis it might be useful to keep the words as they are, for LSA, the questions don't have enough variation in content to benefit for stemming

In [6]:
import re
import nltk
import unidecode

# Regular expression to select all that is not alphabet
# @maybe allow numbers as well
alpha_regex = re.compile('[^a-zA-Z]')

from nltk.corpus import stopwords as sw
if args.lang == "en": stopwords = sw.words('english')
if args.lang == "fr": stopwords = sw.words('french')

from nltk.stem.snowball import SnowballStemmer
if args.lang == "en": stemmer = SnowballStemmer("english")
if args.lang == "fr": stemmer = SnowballStemmer("french")

def tokenize(text):
    '''
        1. Normalized accents
        2. Splits at non alpbhaetic character (@maybe need to revisit for french text)
        3. Caasefolds
    '''    
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            # Handle french accents in text
            word = unidecode.unidecode(word)
            
            # Split at every non alphabet character occurrence
            clean_words = alpha_regex.sub(' ', word).split()
            
            # Casefold
            tokens.extend([word.lower() for word in clean_words])
    
    # Return tokens
    return tokens

def stem(word):
    return stemmer.stem(word).strip()

def preprocess(text):    
    tokenized = tokenize(text)
    # cleaned = [word for word in tokenized if word not in stopwords and word != '']
    # stemmed = [stem(word) for word in cleaned]
    return ' '.join(tokenized)

## Dataset

In [118]:
# Read dataset
dataset = pd.read_csv(args.dataset)
print(f"Dataset has {len(dataset)} documents")

# Detect Language
dataset["detected_lang"] = dataset.question.apply(detect_lang)

print(f"Dataset has {len(dataset[dataset.detected_lang=='en'])} english documents and {len(dataset[dataset.detected_lang=='fr'])} french documents")

Dataset has 5054 documents
Dataset has 2026 english documents and 2726 french documents


In [119]:
# Translation Questions from French to English
if "translated_question" not in dataset.columns:
    dataset["translated_question"] = np.nan

translated_df_path = f"data/{os.path.basename(args.filename)[:-4]}_fr.csv"
fr = pd.DataFrame(columns=["question","translated_question"])
if os.path.exists(translated_df_path):
    fr_csv = pd.read_csv(translated_df_path, index_col=0)
    fr = fr.combine_first(fr_csv)
    dataset.update(fr.translated_question)
    
# Any french questions that need to be translated?
fr_questions = dataset[
                        (dataset.detected_lang == "fr") & 
                        (dataset.translated_question.isnull())
                      ][["question","translated_question"]]

if len(fr_questions)>0:    
    try:
        from googletrans import Translator
        translator = Translator()
        translations = []
        for q in list(fr_questions.question):
            translations.append(translator.translate(q).text)
    except:
        # HACK for when your IP has been blocked
        translations_file_path = f"data/{os.path.basename(args.filename)[:-4]}_fr_translations.txt"
        if os.path.exists(translations_file_path):   
            # Delete if already exists
            os.remove(translations_file_path)
        
        # Create file and initialize empty
        os.mknod(translations_file_path)      
        translations = []
        
        while (len(translations) != len(fr_questions)):
            print(f"Translate the output french to english using gogle translate and copy this into this file - {translations_file_path} \n{'-'*100}")
            print(*fr_questions.question.tolist(),sep="\n")
            # Translate the output french to english using gogle translate 
            # and copy this into a text file save as {filename}_fr_translated.txt in data folder
            input(f"{'-'*100}\nPress Enter to continue...")
            
            with open(translations_file_path,"r") as f:
                translations = f.read().split("\n")  
            
    fr_questions.translated_question = translations
    dataset.update(fr_questions.translated_question)
    dataset[dataset.detected_lang == "fr"][["question","translated_question"]].to_csv(translated_df_path)

Translate the output french to english using gogle translate and copy this into this file - data/covid_questions_2020-04-21_fr_translations.txt 
----------------------------------------------------------------------------------------------------
Je ne me sens pas bien 
À combien de cas sommes-nous? 
Bonjour
Can I run outside
 Bonjour,
Mon chat peut-il attraper la Covid?
Comment me soigner?
Symptômes??
Est-ce que boire de la corona augmente nos chances de contracter le coronavirus?
il y a combien de cas dans la ville de québec du covid 
Quels symptômes de la maladie?
Est ce qu’il y aura un test obligatoire
Est-ce qu'il y aura bientôt un vaccin 
Est-ce que la C
bonjour combien de temps restons malade
Once you recover from covid-19, are you immune? 
Est-ce que la covid-19 va devenir comme l'influenza, un virus qui va circuler selon certaines saisons ?
Is corona virus airborne?
----------------------------------------------------------------------------------------------------
Press Enter 

In [127]:
# Preprocess questions
eng = dataset[dataset.detected_lang == "en"].question.apply(preprocess).to_frame(name="text")
french = dataset[dataset.detected_lang == "fr"].translated_question.apply(preprocess).to_frame(name="text")
dataset["text"] = eng.combine_first(french).dropna()

# Drop
dataset = dataset.dropna(subset=['text'])

print(f"Dataset has {len(dataset)} english documents (translated and otherwise)")

Dataset has 4752 english documents (translated and otherwise)


In [112]:
# dataset[(dataset.language != "en") & (dataset.detected_lang == "en")]

In [113]:
# dataset[(dataset.language == "en") & (dataset.detected_lang != "en")]

## Rules
Note: The order of these rules matters in resolving conflicts

In [128]:
# Statistics
dataset["statistics"] = (
                        dataset.text.str.contains("cases",case=False)|
                        dataset.text.str.contains("death",case=False)|
                        dataset.text.str.contains("died",case=False)|
                        dataset.text.str.contains("mortality rate",case=False)|
                        dataset.text.str.contains("death rate",case=False)|
                        dataset.text.str.contains("deadly",case=False)|
                        dataset.text.str.contains("statistic",case=False)|
                        (
                            dataset.text.str.contains("how",case=False)&
                            dataset.text.str.contains("many",case=False)&
                            dataset.text.str.contains("people",case=False)
                        )
                    ).apply(int)

In [129]:
dataset["animals"] = (
                        dataset.text.str.contains(r"\b(?:animal|bird|cat|dog)s?\b",case=False)
                    ).apply(int)

In [130]:
dataset["caution"] = (
                        dataset.text.str.contains("prevent",case=False)|
                        dataset.text.str.contains("protect",case=False)|
                        dataset.text.str.contains("precaution",case=False)|
                        dataset.text.str.contains("safety",case=False)|
                        (
                            dataset.text.str.contains("keep",case=False)&
                            dataset.text.str.contains("safe",case=False)
                        )
                    ).apply(int)

In [131]:
dataset["viruslife"] = (
                            (
                                dataset.text.apply(fuzzy_match, pattern="(?:covid){e<=2}")|
                                dataset.text.str.contains("corona",case=False)|
                                dataset.text.str.contains("virus",case=False)
                            )&
                            (
                                dataset.text.str.contains("live|stay|survive",case=False)
                            )&
                            (
                                dataset.text.str.contains("on",case=False)
                            )
                    ).apply(int)

In [132]:
dataset["viruskill"] = (
                            (
                                dataset.text.apply(fuzzy_match, pattern="(?:covid){e<=2}")|
                                dataset.text.str.contains("corona",case=False)|
                                dataset.text.str.contains("virus",case=False)
                            )&
                            (
                                dataset.text.str.contains("kills",case=False)
                            )
                    ).apply(int)

In [133]:
dataset["virusfight"] = (
                            (
                                (
                                    dataset.text.apply(fuzzy_match, pattern="(?:covid){e<=2}")|
                                    dataset.text.str.contains("corona",case=False)|
                                    dataset.text.str.contains("virus",case=False)
                                )&
                                (
                                    dataset.text.str.contains("fight",case=False)
                                )&
                                (
                                    dataset.text.str.contains("help",case=False)
                                )
                            )|
                            (
                                dataset.text.str.contains("mask",case=False)|
                                dataset.text.str.contains("glove",case=False)
                            )
                        ).apply(int)

In [134]:
dataset["treatment"] = (
                        dataset.text.str.contains("treatment",case=False)|
                        dataset.text.str.contains("cure",case=False)|
                        dataset.text.str.contains("vaccine",case=False)|
                        dataset.text.str.contains("medic",case=False)
                    ).apply(int)

In [135]:
dataset["incubation"] = (
                        dataset.text.str.contains("incubate",case=False)|
                        dataset.text.str.contains("incubation",case=False)
                    ).apply(int)

In [136]:
dataset["nextsteps"] = (
                        dataset.text.str.contains("i have",case=False) 
                    ).apply(int)

In [137]:
dataset["guideme"] = (
                        dataset.text.str.contains("if",case=False)
                    ).apply(int)

In [138]:
# dataset["hospital"] = (
# #                         dataset.text.str.contains(r"\bgo\b",case=False)&
#                         (
#                             dataset.text.str.contains("hospital",case=False)|                            
#                             dataset.text.str.contains(r"\bER\b",case=False)
#                         )
#                     ).apply(int)

In [139]:
# Dos and Donts
dataset["lockdown"] = (
                        (
                            (
                                dataset.text.str.contains("go (?:on|to|for|out)",case=False)|
                                dataset.text.str.contains("walk",case=False)
                            )&
                            (
                                dataset.text.str.contains("allow",case=False)|
                                dataset.text.str.contains("can",case=False)|
                                dataset.text.str.contains("ok|okay",case=False)|
                                dataset.text.str.contains("should|shall",case=False)
                            )
                        )|
                        (
                            dataset.text.str.contains("lockdown",case=False)|
                            dataset.text.str.contains(r"\bopen\b",case=False)|
                            dataset.text.str.contains(r"\bclose",case=False)
                        )
                    ).apply(int)

In [140]:
dataset["infection"] = (
                        dataset.text.str.contains("infected",case=False)|
                        dataset.text.str.contains("infection",case=False)
                    ).apply(int)

In [141]:
dataset["diff"] = (
                        dataset.text.str.contains("diff",case=False)|
                        dataset.text.apply(fuzzy_match, pattern="(?:distinguish){e<=3}")
                    ).apply(int)

In [142]:
dataset["recover"] = (
                        dataset.text.str.contains("recover",case=False)
                    ).apply(int)

In [143]:
dataset["test"] = (
                        dataset.text.str.contains("tested",case=False)|
                        dataset.text.str.contains("test",case=False)
                    ).apply(int)

In [144]:
dataset["isolation"] = (
                            dataset.text.str.contains(r"\bisolat",case=False)|
                            dataset.text.str.contains(r"\bsocial dist",case=False)
                        ).apply(int)

In [145]:
dataset["dangerisk"] = (
                            dataset.text.str.contains("dangerous",case=False)|
                            dataset.text.str.contains("risk",case=False)
                        ).apply(int)

In [146]:
dataset["transmission"] = (
                            dataset.text.str.contains("transmi",case=False)|
                            dataset.text.str.contains("contract",case=False)|
                            dataset.text.str.contains("spread",case=False)|
                            dataset.text.apply(fuzzy_match, pattern="(?:airborne){e<=3}")
                        ).apply(int)

In [147]:
# Fuzzy Matching of 'Symptom' keyword (accounting for spelling errors)
dataset["symptom"] = (
                        dataset.text.apply(fuzzy_match, pattern="(?:symptom){1<=e<=3}")
                    ).apply(int)

In [148]:
dataset["about"] = (
                        (
                            dataset.text.apply(fuzzy_match, pattern="(?:whats|what (?:is|s))")
                        ) & 
                        (
                            dataset.text.apply(fuzzy_match, pattern="(?:covid){e<=2}")|
                            dataset.text.str.contains("corona",case=False)
                        )
                    ).apply(int)

In [149]:
dataset["future"] = (
                        (
                            (
                                dataset.text.str.contains("how",case=False) &
                                dataset.text.str.contains("long",case=False)
                            )|
                            dataset.text.str.contains("when",case=False)
                        )&
                            dataset.text.str.contains("will",case=False)&
                        (
                            dataset.text.str.contains("last|end|over|normal|done",case=False)
                        )
                    ).apply(int)

In [150]:
dataset["past"] = (
                        (
                            dataset.text.str.contains("how|when|where",case=False) 
                        )&
                            dataset.text.str.contains("did",case=False)&
                        (
                            dataset.text.str.contains("start|begin|began",case=False)
                        )
                    ).apply(int)

In [151]:
dataset = dataset.rename(columns=new_labels)

In [152]:
features = [col_name for col_name in dataset.columns.values.tolist() if "-" in col_name]

In [153]:
features

['situation-stats',
 'covid-animals',
 'personal-caution',
 'covid-life',
 'covid-kill',
 'covid-fight',
 'covid-med',
 'covid-incubation',
 'personal-symptoms',
 'personal-whatif',
 'situation-lockdown',
 'covid-infection',
 'covid-versus',
 'covid-recovery',
 'personal-testing',
 'personal-isolation',
 'covid-contagious',
 'covid-transmission',
 'covid-symptoms',
 'covid-what',
 'situation-future',
 'situation-past']

In [154]:
# Get total number of classes/categories this question qualifies for
dataset["total"] = dataset[features].sum(axis=1)

In [155]:
# Describes how many questions quality for how many classes
dataset.groupby("total")["situation-stats"].describe()["count"]

total
0     1463.0
1     2321.0
2      672.0
3      217.0
4       58.0
5       14.0
6        4.0
7        2.0
11       1.0
Name: count, dtype: float64

In [156]:
# Set default value
dataset["cluster"] = "unclassified"

# For single features
for col in features:
    dataset["cluster"][(dataset.total == 1) & (dataset[col] == True)] = col

/home/designer1/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [157]:
# Resolving multiple classes
for col in features[::-1]:
    dataset["cluster"][(dataset.total > 1) & (dataset[col] == True)] = col

/home/designer1/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [158]:
# # Saving the files
# path = f"output/simple_{args.suffix}_{args.lang}.csv"
# dataset.drop(features, axis="columns").drop(["text","total"], axis="columns").to_csv(path)
# print(f"Rules based output saved to {path}")

In [159]:
# Clustering stats
dataset.groupby("cluster")["question"].count()

cluster
covid-animals           53
covid-contagious       154
covid-fight             80
covid-incubation        28
covid-infection         98
covid-kill               6
covid-life             147
covid-med              145
covid-recovery          23
covid-symptoms         518
covid-transmission     124
covid-versus             3
covid-what             143
personal-caution       126
personal-isolation      57
personal-symptoms      508
personal-testing       219
personal-whatif        477
situation-future        40
situation-lockdown     153
situation-past           4
situation-stats        183
unclassified          1463
Name: question, dtype: int64

### Length analysis for situations

In [228]:
dataset["len"] = dataset.text.apply(lambda x: len(x.split()))

## LSA and AHC

In [260]:
eng = dataset[dataset.detected_lang == "en"].question.to_frame(name="question")
french = dataset[dataset.detected_lang == "fr"].translated_question.to_frame(name="question")
d = eng.combine_first(french).dropna()

In [261]:
d["cluster"] = dataset.cluster
# Adding length as a feature
d["len"] = dataset.len

In [262]:
d = d[d.cluster=="unclassified"]

In [263]:
import nltk
stopwords_list = nltk.corpus.stopwords.words('english')

In [264]:
wn_lemmatizer = nltk.stem.WordNetLemmatizer()
def bulk_tokenizer(texts):
#      return [[wn_lemmatizer.lemmatize(token) for token in nltk.word_tokenize(text)] for text in texts]
     return [nltk.word_tokenize(text) for text in texts]

In [265]:
args.n_topics = 15
args.dist_thresh = 1.2
model = utils.text.representation.LSI(args, tokenizer=bulk_tokenizer)
d["embedding"] = model.generate_embedding(d.question, returnarray=False)

features = pd.DataFrame(d["embedding"].values.tolist(), index= d.index).to_numpy()

In [266]:
use_len = False
if use_len:
    # Add len to the feature vector
    d[["embedding","len"]].apply(lambda x: x.embedding.append(x.len), axis=1)

    from sklearn import preprocessing

    x = d.embedding.tolist() #returns a numpy array
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    features = x_scaled

In [267]:
# Cluster
X = features
clustering = AgglomerativeClustering(n_clusters=None, compute_full_tree=True, distance_threshold=args.dist_thresh).fit(X)
d["ahc_label"] = clustering.labels_

# Misc.
args.n_clusters = len(d["ahc_label"].unique())
print(f"Found {args.n_clusters} clusters")
d.groupby("ahc_label")["question"].count().sort_values()

Found 38 clusters


ahc_label
33      4
32      6
27      7
31      8
20     10
14     11
29     11
10     12
3      12
26     15
22     15
5      16
25     18
0      18
24     19
23     19
13     20
4      24
17     26
21     27
6      31
18     33
19     40
12     43
8      43
37     48
28     49
35     51
16     55
30     55
1      60
7      66
9      69
36     70
11     76
15     77
34     88
2     211
Name: question, dtype: int64

In [268]:
d.groupby("ahc_label")["question"].count().describe()

count     38.000000
mean      38.500000
std       37.205601
min        4.000000
25%       15.000000
50%       26.500000
75%       54.000000
max      211.000000
Name: question, dtype: float64

In [269]:
for label in d.ahc_label.unique():
    print(f"\ncluster #{label}, count - {len(d[d.ahc_label==label])}")
    print(d[d.ahc_label==label][:10].question.tolist())


cluster #37, count - 48
["What is Dialogue's position on sick notes?", 'What is the danger for a newborn baby?', 'What is the% of the population affected by each country in the world?', "What's happen for country near the Ecuador with a lot of hot? ", 'What is the contagion period?', 'What is the propagation time', 'What is the case fatality rate?', "What's the current count in India", 'What is the advice surrounding outdoor exercise/sport?', 'What is the situation right now in the world?']

cluster #1, count - 60
['What are Dialogue services?', 'What are the complications of the disease?', 'What are the instructions to follow?', 'what are the conditions under which people are most likely to develop complications versus those who won’t?  ', 'What are the dangers for people over 70?', 'Canadians are caught in Peru, are repatriation flights planned in this country?', 'What are the rates of hospitalization and the need for intensive care', 'Which regions of Montreal are most affected?', 

In [272]:
if "ahc_label" in dataset.columns:
    dataset = dataset.drop(columns=["ahc_label"])
dataset = dataset.join(d["ahc_label"])

In [273]:
path = f"output/simpleLsa_{args.n_topics}n{args.dist_thresh}dt_{args.suffix}.csv"
dataset.to_csv(path)
print(f"AHC on top of rule based output saved to {path}")

AHC on top of rule based output saved to output/simpleLsa_15n1.2dt_2020-04-21.csv


## LSA and AHC - 2nd Time

In [221]:
path = f"output/simpleLsa_{args.n_topics}n{args.dist_thresh}dt_{args.suffix}_{args.lang}_checked.csv"
print(path)

output/simpleLsa_15n0.9dt_2020-04-14_fr_checked.csv


In [222]:
d = pd.read_csv(path, index_col=0)

In [223]:
d = d[["question","text", "translation", "cluster", "tags", "ahc_label", "cluster_count"]]

In [224]:
d.head()

,question,text,translation,cluster,tags,ahc_label,cluster_count
247,Les animaux peuvent-ils transmettre le COVID-19?,les animaux peuvent ils transmettre le covid,Can animals transmit COVID-19?,covid-animals,covid-animals,NaN,NaN
360,"Mon chat reste libre de ces mouvements, peut-i...",mon chat reste libre de ces mouvements peut il...,"My cat remains free from these movements, can ...",covid-animals,covid-animals,NaN,NaN
611,Les animaux (sauvages ou domestiques) peuvent-...,les animaux sauvages ou domestiques peuvent il...,Can animals (wild or domestic) contract the vi...,covid-animals,covid-animals,NaN,NaN
621,Combine de temps le virus peut rester sur le p...,combine de temps le virus peut rester sur le p...,How long can the virus stay on the hair of ani...,covid-animals,"covid-animals, covid-life",NaN,NaN
714,Est-ce que mon chat ou mon chien peuvent attra...,est ce que mon chat ou mon chien peuvent attra...,Can my cat or dog catch covid-19?,covid-animals,covid-animals,NaN,NaN


In [225]:
d = create_rules_based_clusters(d, cluster_col="cluster_2")

/home/designer1/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [226]:
d.head()

,question,text,translation,cluster,tags,ahc_label,cluster_count,cluster_2
247,Les animaux peuvent-ils transmettre le COVID-19?,les animaux peuvent ils transmettre le covid,Can animals transmit COVID-19?,covid-animals,covid-animals,NaN,NaN,covid-animals
360,"Mon chat reste libre de ces mouvements, peut-i...",mon chat reste libre de ces mouvements peut il...,"My cat remains free from these movements, can ...",covid-animals,"covid-animals, covid-life",NaN,NaN,unclassified
611,Les animaux (sauvages ou domestiques) peuvent-...,les animaux sauvages ou domestiques peuvent il...,Can animals (wild or domestic) contract the vi...,covid-animals,covid-animals,NaN,NaN,covid-animals
621,Combine de temps le virus peut rester sur le p...,combine de temps le virus peut rester sur le p...,How long can the virus stay on the hair of ani...,covid-animals,"covid-animals, covid-life",NaN,NaN,unclassified
714,Est-ce que mon chat ou mon chien peuvent attra...,est ce que mon chat ou mon chien peuvent attra...,Can my cat or dog catch covid-19?,covid-animals,covid-animals,NaN,NaN,covid-animals


In [333]:
# import nltk
# stopwords_list = nltk.corpus.stopwords.words('english')
# wn_lemmatizer = nltk.stem.WordNetLemmatizer()

In [334]:
def bulk_tokenizer(texts):
#      return [[wn_lemmatizer.lemmatize(token) for token in nltk.word_tokenize(text)] for text in texts]
     return [text.split() for text in texts]

In [335]:
args.n_topics = 15
args.dist_thresh = 0.8
model = utils.text.representation.LSI(args, tokenizer=bulk_tokenizer)
d["embedding"] = model.generate_embedding(d.text, returnarray=False)

# Cluster
X = pd.DataFrame(d["embedding"].values.tolist(), index= d.index).to_numpy()
clustering = AgglomerativeClustering(n_clusters=None, compute_full_tree=True, distance_threshold=args.dist_thresh).fit(X)
d["ahc_label"] = clustering.labels_

# Misc.
args.n_clusters = len(d["ahc_label"].unique())
print(f"Found {args.n_clusters} clusters")
d.groupby("ahc_label")["text"].count().sort_values()

Found 53 clusters


ahc_label
51     3
40     3
48     4
18     5
43     5
34     6
10     6
31     7
33     7
25     8
47     8
22     8
11     8
36     9
37     9
41    10
2     10
44    11
20    11
45    11
5     11
29    12
46    13
4     13
52    13
39    14
42    18
49    19
28    20
12    22
35    22
30    22
26    25
27    25
23    26
14    26
38    28
24    28
7     28
15    30
0     31
17    31
19    33
16    33
32    33
8     34
13    37
21    39
3     40
50    40
9     50
1     54
6     65
Name: text, dtype: int64

In [336]:
d.groupby("ahc_label")["text"].count().describe()

count    53.000000
mean     20.452830
std      14.209377
min       3.000000
25%       9.000000
50%      18.000000
75%      30.000000
max      65.000000
Name: text, dtype: float64

In [337]:
# To Translated
# print(*d.question.tolist(),sep="\n")

In [338]:
for label in d.ahc_label.unique():
    print(f"\ncluster #{label}, count - {len(d[d.ahc_label==label])}")
    print(*zip(d[d.ahc_label==label][:10].question.tolist(),d[d.ahc_label==label][:10].text.tolist()),sep="\n")


cluster #27, count - 25
('Est ce que la toux est sèche ou grasse?', 'est ce que la toux est seche ou grasse')
('Est ce que la question: arrivez vous de voyage, est ', 'est ce que la question arrivez vous de voyage est')
("Est ce que la question, arrivez vous de voyage est encore d'actualite?  Messeble que la propagation est au dela de ca? ", 'est ce que la question arrivez vous de voyage est encore d actualite messeble que la propagation est au dela de ca')
('Est-ce que la transmission communautaire est débuté?', 'est ce que la transmission communautaire est debute')
('Est ce que l’assurance maladie sera valide à l’extérieur de la province?', 'est ce que l assurance maladie sera valide a l exterieur de la province')
('Quand est-ce que ça va être terminé?', 'quand est ce que ca va etre termine')
('Est-ce que ca donne la diarrhée ?', 'est ce que ca donne la diarrhee')
('Est-ce vrai que 75% de la population sera infecté de toutes façons? ', 'est ce vrai que de la population sera infecte 

In [339]:
dataset.head()

,timestamp_est,anonymous_id,language,question,detected_lang,text,translation,statistics,caution,animals,...,guideme,test,isolation,dangerisk,transmission,symptom,about,future,total,cluster
3,2020-03-19T14:00:19.88Z,71a29314-2b32-43d5-b09a-1795a5380e60,fr,LP TEST question,fr,lp test question,LP TEST question,0,0,0,...,0,1,0,0,0,0,0,0,1,test
6,2020-03-19T13:47:48.848Z,0b0df84f-dcc5-42c0-980e-bafacd3bbdbe,fr,"Je fais une sarcoidose pulmonaire, je suis tu ...",fr,je fais une sarcoidose pulmonaire je suis tu p...,"I have pulmonary sarcoidosis, I'm more at risk...",0,0,0,...,0,0,0,1,0,0,0,0,1,dangerisk
11,2020-03-19T15:39:52.508Z,b3cd859f-a9f3-4985-a884-0a8387a9daa4,fr,Est ce que la toux est sèche ou grasse?,fr,est ce que la toux est seche ou grasse,Is the cough dry or oily?,0,0,0,...,0,0,0,0,0,0,0,0,0,unclassified
12,2020-03-19T14:16:03.429Z,34327a70-7e54-4f53-bdf6-6809c83a8d5e,fr,Je ne me sens pas bien,fr,je ne me sens pas bien,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,unclassified
17,2020-03-19T13:55:41.747Z,59eafb5a-76c4-4e07-891d-6e79276fa82d,fr,"Bonjour,",fr,bonjour,"Bonjour,",0,0,0,...,0,0,0,0,0,0,0,0,0,unclassified


In [340]:
dataset = dataset.drop(features, axis="columns").drop("total", axis="columns").join(d["ahc_label"])

In [341]:
dataset.head()

,timestamp_est,anonymous_id,language,question,detected_lang,text,translation,cluster,ahc_label
3,2020-03-19T14:00:19.88Z,71a29314-2b32-43d5-b09a-1795a5380e60,fr,LP TEST question,fr,lp test question,LP TEST question,test,NaN
6,2020-03-19T13:47:48.848Z,0b0df84f-dcc5-42c0-980e-bafacd3bbdbe,fr,"Je fais une sarcoidose pulmonaire, je suis tu ...",fr,je fais une sarcoidose pulmonaire je suis tu p...,"I have pulmonary sarcoidosis, I'm more at risk...",dangerisk,NaN
11,2020-03-19T15:39:52.508Z,b3cd859f-a9f3-4985-a884-0a8387a9daa4,fr,Est ce que la toux est sèche ou grasse?,fr,est ce que la toux est seche ou grasse,Is the cough dry or oily?,unclassified,27.0
12,2020-03-19T14:16:03.429Z,34327a70-7e54-4f53-bdf6-6809c83a8d5e,fr,Je ne me sens pas bien,fr,je ne me sens pas bien,NaN,unclassified,4.0
17,2020-03-19T13:55:41.747Z,59eafb5a-76c4-4e07-891d-6e79276fa82d,fr,"Bonjour,",fr,bonjour,"Bonjour,",unclassified,31.0


In [342]:
# print(*dataset.question.tolist(),sep="\n")

In [343]:
path = f"output/simpleLsa_{args.n_topics}n{args.dist_thresh}dt_{args.suffix}_{args.lang}.csv"
dataset.to_csv(path)
print(f"AHC on top of rule based output saved to {path}")

AHC on top of rule based output saved to output/simpleLsa_15n0.8dt_2020-03-27T19_12_30.866993Z_fr.csv


## Iterative Clustering